In [9]:
%%time

from datetime import datetime
import xarray as xr
import os
import sys

opath = '/glade/work/nijssen/scratch'

hru_id_files = {'attributes':'/glade/work/andywood/wreg/gis/shapes/cali_huc12/cali_hrus.txt',
                'parameters':'/glade/work/andywood/wreg/gis/shapes/cali_huc12/cali_hrus.txt'}

gru_id_files = hru_id_files

ifiles = {'attributes':'/glade/work/gangrade/R18_HUC12_summa/R18_HUC12_attribute.nc',
          'parameters':'/glade/work/gangrade/R18_HUC12_summa/R18_HUC12_ParamTrial_new.nc'}


for key in ifiles:
    hru_id_file = hru_id_files[key]
    gru_id_file = gru_id_files[key]
    ifile = ifiles[key]
    ofile = os.path.join(opath, os.path.basename(ifile.replace('.nc', '_subset.nc')))

    with open(hru_id_file) as f:
        hru_ids = [int(x) for x in f]

    with open(gru_id_file) as f:
        gru_ids = [int(x) for x in f]


    ds_org = xr.open_dataset(ifile)
    ds_hru = ds_org.drop_dims('gru')
    ds_gru = ds_org.drop_dims('hru')
    
    # hack: if gruId does not exist, copy it from the hruId
    if 'gruId' not in ds_gru:
        ds_gru['gruId'] = xr.Variable(ds_gru.dims, ds_org.hruId.values)
        
    ds_hru_subset = ds_hru.where(ds_hru.hruId.isin(hru_ids), drop=True)
    ds_gru_subset = ds_gru.where(ds_gru.gruId.isin(gru_ids), drop=True)
        
    ds_subset = ds_hru_subset.merge(ds_gru_subset)
    
    # Write hru IDs from the ID file that were not in the NetCDF file to stdout
    missing = set(hru_ids).difference(set(ds_hru.hruId.values))
    if missing:
        print("Missing hru IDs: ")
        for x in missing:
            print(x)

    # Write gru IDs from the ID file that were not in the NetCDF file to stdout
    missing = set(gru_ids).difference(set(ds_gru.gruId.values))
    if missing:
        print("Missing gru IDs: ")
        for x in missing:
            print(x)
            
    # hack: drop the gru ID if it was not in the original file
    if 'gruId' not in ds_org:
        ds_subset = ds_subset.drop('gruId')
    
    # make sure that the subsetted types are the same as the original ones
    for var in ds_subset.variables:
        ds_subset[var] = ds_subset[var].astype(ds_org[var].dtype)

    # update the history attribute
    history = '{}: {}\n'.format(datetime.now().strftime('%c'), 
                                ' '.join(sys.argv))
    if 'history' in ds_subset.attrs:
        ds_subset.attrs['history'] = history + ds_subset.attrs['history']
    else:
        ds_subset.attrs['history'] = history
        
    ds_subset.to_netcdf(os.path.join(opath, ofile))

CPU times: user 134 ms, sys: 6.56 ms, total: 141 ms
Wall time: 151 ms


In [10]:
%%time

# subsetting the cold state file

from datetime import datetime
import xarray as xr
import os
import sys

opath = '/glade/work/nijssen/scratch'

hru_id_file = '/glade/work/andywood/wreg/gis/shapes/cali_huc12/cali_hrus.txt'

ifile = '/glade/work/gangrade/R18_HUC12_summa/R18_HUC12_InitialState.nc'

ofile = os.path.join(opath, os.path.basename(ifile.replace('.nc', '_subset.nc')))

with open(hru_id_file) as f:
    hru_ids = [int(x) for x in f]

ds = xr.open_dataset(ifile)

ds_subset = ds.where(ds.hruID.isin(hru_ids), drop=True)

# Write hru IDs from the ID file that were not in the NetCDF file to stdout
missing = set(hru_ids).difference(set(ds.hruID.values))
if missing:
    print("Missing hru IDs: ")
    for x in missing:
        print(x)

# make sure that the subsetted types are the same as the original ones
for var in ds_subset.variables:
    ds_subset[var] = ds_subset[var].astype(ds[var].dtype)
    
# update the history attribute
history = '{}: {}\n'.format(datetime.now().strftime('%c'), 
                            ' '.join(sys.argv))
if 'history' in ds_subset.attrs:
    ds_subset.attrs['history'] = history + ds_subset.attrs['history']
else:
    ds_subset.attrs['history'] = history

ds_subset.to_netcdf(os.path.join(opath, ofile))

CPU times: user 48.9 ms, sys: 9 ms, total: 57.9 ms
Wall time: 271 ms
